In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn


In [16]:
import qns3vm


In [17]:
dir(qns3vm)

['ALLOW_THREADS',
 'AxisError',
 'BUFSIZE',
 'CLIP',
 'ComplexWarning',
 'DataSource',
 'DictLinearKernel',
 'DictRBFKernel',
 'ERR_CALL',
 'ERR_DEFAULT',
 'ERR_IGNORE',
 'ERR_LOG',
 'ERR_PRINT',
 'ERR_RAISE',
 'ERR_WARN',
 'FLOATING_POINT_SUPPORT',
 'FPE_DIVIDEBYZERO',
 'FPE_INVALID',
 'FPE_OVERFLOW',
 'FPE_UNDERFLOW',
 'False_',
 'Inf',
 'Infinity',
 'LinearKernel',
 'MAXDIMS',
 'MAY_SHARE_BOUNDS',
 'MAY_SHARE_EXACT',
 'MachAr',
 'ModuleDeprecationWarning',
 'NAN',
 'NINF',
 'NZERO',
 'NaN',
 'PINF',
 'PZERO',
 'QN_S3VM',
 'QN_S3VM_Dense',
 'QN_S3VM_Sparse',
 'RAISE',
 'RBFKernel',
 'RankWarning',
 'SHIFT_DIVIDEBYZERO',
 'SHIFT_INVALID',
 'SHIFT_OVERFLOW',
 'SHIFT_UNDERFLOW',
 'ScalarType',
 'TooHardError',
 'True_',
 'UFUNC_BUFSIZE_DEFAULT',
 'UFUNC_PYVALS_NAME',
 'VisibleDeprecationWarning',
 'WRAP',
 '_UFUNC_API',
 '__author__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__version__',
 '_add_newdoc_ufunc',
 '

In [ ]:
df_train = pd.read_csv('train.csv')

In [ ]:
df_train.head(5)

In [ ]:
df_train.shape

In [ ]:
df_train['wheezy-copper-turtle-magic'].unique()

In [ ]:
cols = [c for c in df_train.columns if c not in ['id', 'target', 'wheezy-copper-turtle-magic']]
au=['wheezy-copper-turtle-magic']
tar = ['target']

In [ ]:
from sklearn.svm import SVC
import umap
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from lightgbm import LGBMClassifier
from sklearn.svm import NuSVC

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

class Classifier:
    def __init__(self, n_red = 10, dim_red = True):
        self.dim_red = dim_red
        self.n_red = n_red
        self.supervised = False
        ## Select a classifier
#         self.clf = SVC(probability=True,kernel='poly',degree=4,gamma='auto')
#         self.clf = LGBMClassifier(n_estimators=1000, n_jobs = 8)
#         self.clf = GradientBoostingClassifier(n_estimators=100)
#         self.clf = xgb.XGBClassifier(n_estimators = 100, n_jobs = 8)
#         self.clf = KNeighborsClassifier(n_neighbors=15)
#         self.clf = RandomForestClassifier(n_estimators=100, n_jobs=8)
        self.clf = NuSVC(probability=True, kernel='poly', degree=4, gamma='auto', random_state=4, nu=0.6, coef0=0.08)
        
        ## Select a dimension reduction method
#         self.mapper = [PCA(n_components=47, random_state=4)]#,StandardScaler()]
#         self.mapper = [VarianceThreshold(threshold=1.5)]
#         self.mapper = umap.UMAP(n_components = self.n_red,n_neighbors = 15)
    
    def fitting(self, X, y, X_test):
        embeddings = np.concatenate((X, X_test), axis = 0)
        if self.dim_red:
            if self.supervised:
                self.mapper = self.mapper.fit(X, y=y)
            else:
                temp = []
                for elt in self.mapper:
                    temp.append(elt.fit(embeddings))
                    self.mapper = temp
            embeddings = X
            for elt in self.mapper:
                embeddings = elt.transform(embeddings)
        else:
            embeddings = X
            
        self.clf.fit(embeddings, y)
        
    def predicting(self, X):
        embeddings = X
        if self.dim_red:
            for elt in self.mapper:
                embeddings = elt.transform(embeddings)

        return self.clf.predict_proba(embeddings)[:,1]

In [ ]:
min1 = 0
max1 = 512
X= df_train[(df_train['wheezy-copper-turtle-magic']<max1)&(df_train['wheezy-copper-turtle-magic']>=min1)][cols+tar+au]
y = X['target'].values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 43)
df = X_train